In [ ]:
from collections import UserDict
import re
import datetime

class Field:
    """
    Base class for fields in a contact record.
    """
    def __init__(self, value):
        self.value = value
    
    def __str__(self):
        return str(self.value)

class Name(Field):
    """
    Class for contact name with validation.
    """
    def __init__(self, name):
        super().__init__(Name.__validate_name(name))
    
    @staticmethod
    def __validate_name(name: str) -> str:
        """
        Validate the name to contain only letters and spaces.
        :param name: str The input name.

        Return: Cleaned name if valid, raise an error otherwise.
        """
        clean_name = name.strip().replace("  "," ")
        if not clean_name.isalpha():
            clean_name = re.findall(r"[A-Za-zА-Яа-яЁёЇїІіЄєҐґ\s]+", clean_name)
            if isinstance(clean_name, list) and clean_name:
                clean_name = clean_name[0]
            if clean_name:
                return clean_name
        raise ValueError("Invalid name format. Name should contain only letters and spaces")
    

class Phone(Field):
    """
    Class for contact phone number with validation.
    """
    def __init__(self, number: str):
        super().__init__(Phone.__normalize_phone(number))
    
    def __eq__(self, other):
        return isinstance(other, Phone) and self.value == other.value

    @staticmethod
    def __normalize_phone(phone_number: str) -> str:
        """
        Normalize a phone number to the format +380XXXXXXXXX.
        :param phone_number: str The input phone number in various formats.

        Return: str The normalized phone number or an empty string if invalid.
        """
        try:
            phone_number = phone_number.strip().replace(" ","")
            phone_number = phone_number[phone_number.index("0"):]
            found_number = re.fullmatch(r"(\d{3})[\s\t()\n-]*(\d{3})[\s\t()\n-]*(\d{2})[\s\t()\n-]*(\d{2})", phone_number)
            found_number = f"+38{found_number[0]}"
            _ = found_number[12]  # Test if the string is long enough
            return found_number
        except:
            raise ValueError("Invalid phone number format. Expected format: +380-XXX-XXX-XX-XX.")

class Birthday(Field):
    def __init__(self, value):
        super().__init__(Birthday.__validate_birthday(value))
    
    @staticmethod
    def __validate_birthday(birthday_str: str) -> datetime.date:
        """
        Validate and convert birthday string to a date object.
        :param birthday_str: str The input birthday string in DD.MM.YYYY format.

        Return: datetime.date The validated birthday date.
        """
        try:
            day, month, year = map(int, birthday_str.split('.'))
            return datetime.date(year=year, month=month, day=day)
        except ValueError:
            raise ValueError("Invalid date format. Use DD.MM.YYYY")

class Record:
    """
    Class representing a contact record.
    """
    def __init__(self, name):
        self.name = Name(name)
        self.phones = []
        self.birthday = None

    def add_phone(self, number):
        phone = Phone(number)
        if phone in self.phones:
            raise ValueError("Phone number already exists")
        self.phones.append(phone)

    def delete_phone(self, number):
        phone = self.find_phone(number)
        if not phone:
            raise ValueError("Phone number not found")
        self.phones.remove(phone)

    def edit_phone(self, old_number, new_number):
        if self.find_phone(old_number):
            if self.find_phone(old_number) != Phone(new_number):
                self.delete_phone(old_number)
                self.add_phone(new_number)
            else:
                raise ValueError("New phone number already exists")
        else:
            raise ValueError("Phone number not found")

    def find_phone(self, number):
        phone = Phone(number)
        if phone in self.phones:
            return phone
        return None

    def add_birthday(self, birthday_str):
        pass

    def __str__(self):
        return f"Contact name: {self.name.value}, phones: {'; '.join(p.value for p in self.phones)}"

class AddressBook(UserDict):
    """
    Class representing an address book.
    """
    def add_record(self, record: Record):
        self.data[record.name.value] = record

    def find_record(self, name):
        return self.data.get(name)

    def delete_record(self, name):
        if name in self.data:
            del self.data[name]
        else:
            raise KeyError("Contact not found")
    
    # Оновити методи для роботи з контактами
    def get_upcoming_birthdays(self) -> list:
        """
        Get a list of upcoming birthdays within the next 7 days. If the day is a weekend, move it to the next Monday.

        Return: List of dictionaries with 'name' and 'congratulation_date' (%Y.%m.%d) keys.
        """
        # Define date boundaries
        now_date = datetime.datetime.now().date()
        treshold_date = now_date + datetime.timedelta(days=7)
        celebrations = []

        # Check all users and prepare their celebration dates
        for user in self.data.values():
            # Skip users without birthday
            if not user['birthday']:
                continue
            # Calculate this year's and next year's (if this year has passed) birthday dates
            this_year_birthday = user['birthday'].date().replace(year=datetime.datetime.now().year)
            next_year_birthday = this_year_birthday.replace(year=datetime.datetime.now().year + 1)

            # If birthday is during this year within the next 7 days
            if now_date <= this_year_birthday <= treshold_date:
                if this_year_birthday.weekday() in (5, 6):
                    celebration_date = this_year_birthday + datetime.timedelta(days=(7 - this_year_birthday.weekday())) # Move to next Monday
                else:
                    celebration_date = this_year_birthday
                celebrations.append({'name': user['name'], 'congratulation_date': celebration_date.strftime("%Y.%m.%d")})
            # If birthday has already passed this year, check next year's birthday
            elif now_date > this_year_birthday and next_year_birthday <= treshold_date:
                if next_year_birthday.weekday() in (5, 6):
                    celebration_date = next_year_birthday + datetime.timedelta(days=(7 - next_year_birthday.weekday())) # Move to next Monday
                else:
                    celebration_date = next_year_birthday
                celebrations.append({'name': user['name'], 'congratulation_date': celebration_date.strftime("%Y.%m.%d")})
            # Continue searching
            else:
                continue
        return celebrations 


In [ ]:
book = AddressBook()

In [69]:
phone_number = "  +380996593412  "
phone_number = phone_number.strip().replace(" ","")

try:
    phone_number = phone_number[phone_number.index("0"):]
    found_number = re.fullmatch(r"(\d{3})[\s\t()\n-]*(\d{3})[\s\t()\n-]*(\d{2})[\s\t()\n-]*(\d{2})", phone_number)
    found_number = f"+38{found_number[0]}"
    _ = found_number[12]  # Test if the string is long enough

except:
    raise ValueError("Invalid phone number format. Expected format: +380-XXX-XXX-XX-XX.")


In [70]:
found_number

'+380996593412'